This tutorial demonstrates data processing steps specifically as applied to scanning electron diffraction (SED) datasets acquired at the Electron Physical Sciences Imaging Centre (ePSIC) using the Merlin-Medipix detector system installed on E02. Helfpul code snippets are provided to deal with the file structures and formats associated with this system.

# Change Log

25/03/18 Duncan Johnstone - Developed for ePSIC HyperSpy Workshop

# Requirements

pyXem 0.5

HyperSpy 1.3

PyMatGen

# Materials & Methods

Data used in this useage example was acquired from a calibration standard Au cross-grating (Agar Scientific) 

# Contents

1. <a href='#loa'> Loading & Data Conversion</a>
2. <a href='#vdf'> Inspection & Virtual Diffraction Imaging</a>
3. <a href='#vec'> Peak Finding & Vector Analysis</a>

Import pyXem and other required libraries

In [1]:
%matplotlib tk
import pyxem as pxm
import numpy as np
import os

# <a id='loa'></a> 1. Loading and Data Conversion

Create a list of file names for iteration by walking through timestamped directories

In [2]:
filenames = []
for root, dirs, files in os.walk("./demo_mib_data/"):
    for file in files:
        if file.endswith(".hdr"):
             filenames.append(os.path.join(root, file))

In [6]:
filenames

['./demo_mib_data/20171210 112111/AuCalibration_10umCLap_15cmL_200kX_0_5.hdr',
 './demo_mib_data/20171210 111859/AuCalibration_10umCLap_15cmL_150kX_0_5.hdr']

Define corresponding lists of calibration values based on experimental conditions

Select the 

In [ ]:
fname = filenames[0]
tstamp = os.path.dirname(fname)[-6:]

In [ ]:
dp = pxm.load_mib(fname, scan_size=256)
    #crop center
    roi = pxm.roi.RectangularROI(left=103, top=103, right=153, bottom=153)
    dpc = roi(dp, axes=dp.axes_manager.signal_axes)
    #perform alignment
    centers = dpc.get_direct_beam_position(sigma=3)
    shifts = centers.data - np.array((25,25))
    shifts = shifts.reshape(65025, 2)
    dp.align2D(shifts=shifts, crop=False, fill_value=0)

In [4]:
i=0
for fname in filenames:
    tstamp = os.path.dirname(fname)[-6:]
    #load data
    dp = pxm.load_mib(fname, scan_size=256)
    #crop center
    roi = pxm.roi.RectangularROI(left=103, top=103, right=153, bottom=153)
    dpc = roi(dp, axes=dp.axes_manager.signal_axes)
    #perform alignment
    centers = dpc.get_direct_beam_position(sigma=3)
    shifts = centers.data - np.array((25,25))
    shifts = shifts.reshape(65025, 2)
    dp.align2D(shifts=shifts, crop=False, fill_value=0)
    #set calibrations
    dp.set_diffraction_calibration(0.00889)
    dp.set_scan_calibration(5.0)
    #save data
    dp.save('./epsic_eg_' + tstamp)
    #summed diffraction
    dpeg = dp.sum((0,1))
    dpeg.change_dtype('float32')
    dpeg.save('./epsic_eg_dpeg_' + tstamp + '.tif')
    #vbf image
    circ = pxm.roi.CircleROI(cx=0, cy=0, r=10*0.00889)
    vbf = dp.get_virtual_image(circ)
    vbf = vbf.as_signal2D((0,1))
    vbf.change_dtype('float32')
    vbf.save('./epsic_eg_vbf_' + tstamp + '.tif')
    #vdf image
    ann = pxm.roi.CircleROI(cx=0, cy=0, r=100*0.00889, r_inner=50*0.00889)
    vdf = dp.get_virtual_image(ann)
    vdf = vdf.as_signal2D((0,1))
    vdf.change_dtype('float32')
    vdf.save('./epsic_eg_vdf_' + tstamp + '.tif')
    i=i+1

A Jupyter Widget

A Jupyter Widget

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/hremadmin/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/hremadmin/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/hremadmin/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



A Jupyter Widget

A Jupyter Widget

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/home/hremadmin/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/hremadmin/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/hremadmin/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



# 2. <a href='#vdf'></a> Inspection & Virtual Diffraction Imaging

In [5]:
dp = pxm.load('./epsic_eg_111859.hspy')

# 3. <a href='#vec'> </a> Peak Finding & Vector Analysis